In [1]:
# Import our SparkSession so we can use it
from pyspark.sql import SparkSession, SQLContext, functions as F
from pyspark.sql.functions import col

# Create our SparkSession, this can take a couple minutes locally
spark = SparkSession.builder.appName("Business_review_JSON").config('spark.sql.broadcastTimeout',-1).getOrCreate()

In [6]:
df_business = spark.read.json("../data_source/business.json")
df_reviews = spark.read.json("../data_source/review.json")

In [ ]:
df_business.printSchema()

In [25]:
df_reviews.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [14]:
States_Counting = df_business.groupBy("state").count()
States_Counting.orderBy(col("count").desc()).show(5)

+-----+-----+
|state|count|
+-----+-----+
|   AZ|56686|
|   NV|36312|
|   ON|33412|
|   NC|14720|
|   OH|14697|
+-----+-----+
only showing top 5 rows



In [ ]:
df_AZ_business = df_business.filter(col("state") == "AZ")

In [30]:
df_business_review = df_AZ_business.alias("business").join(df_reviews.alias("review"), "business_id", "inner") \
    .select("business.*", "review.*")

In [31]:
# Save the join in a new json file...

In [38]:
df_business_review.selectExpr("business.*", 
                              "review.review_id",
                              "review.user_id",
                              "review.text",
                              "review.stars as review_stars") \
    .coalesce(1).write.format('json').save('../data_source/business_review')



Py4JJavaError: An error occurred while calling o635.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc

In [20]:
df_business.groupBy("business_id", "stars").count().show()


+--------------------+-----+-----+
|         business_id|stars|count|
+--------------------+-----+-----+
|-I06hkMFrX0KBqu61...|  5.0|    3|
|74LU6K2ro5AQXKT0J...|  4.0|   16|
|7EnzO970Zdrd_gjIJ...|  4.5|   19|
|93yfzr8B1s-mhan2W...|  3.0|    6|
|l3DOX58AX5mcHqnbJ...|  3.5|    6|
|xdBeEql6sCe7zBHZx...|  5.0|    7|
|FYR3nVpNvY2X6dBu1...|  5.0|    3|
|IVRpIYUnWpvSneeGH...|  5.0|   10|
|L7onn3bgx-8HgKERS...|  5.0|   12|
|dtp3UPOH3Fig5zW2t...|  3.5|    3|
|lcqzUoxA6E2aVIF7_...|  1.0|    3|
|UBOUVw45T86I6FgXA...|  5.0|    4|
|UcCJfq0PElINpflaT...|  2.5|   14|
|XetqQFNRm_lqfNZ6Y...|  2.5|    7|
|ZdF_p8ff2pNuZcX_A...|  3.5|    3|
|lgdde-A-rnzOw9ybm...|  3.5|   59|
|qzPW50oW2TN1yG8XK...|  3.5|    3|
|rDpX16jDGa4YGiycp...|  4.0|    3|
|z5QStGLj6ua31xXnN...|  5.0|    6|
|412zay6TFdL20X764...|  2.5|    6|
+--------------------+-----+-----+
only showing top 20 rows



In [36]:
df_business = df_business.orderBy(df_business.state.desc())


+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------+----------+-----------+--------------------+-----------+---------+------------+--------------------+-----+-----+--------------------+--------------------+
|             address|          attributes|         business_id|          categories|      city|               hours|is_open|  latitude|  longitude|                name|postal_code|rev_stars|review_count|           review_id|stars|state|                text|             user_id|
+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------+----------+-----------+--------------------+-----------+---------+------------+--------------------+-----+-----+--------------------+--------------------+
|     61 Queen Street|[,, 'none', {'rom...|FByZsT1Sob5Vf1AYJ...|Indian, Pakistani...|     Leeds|                null|      1|43.6528212|-79.3763454|         Des

In [50]:
df_business.filter(df_business.state == "GA").groupBy("city").count().show(200)


+---------+-----+
|     city|count|
+---------+-----+
| Avondale|   11|
|Las Vegas|    3|
+---------+-----+



In [5]:
df_business.filter((col("rev_stars") >= 4 ) & (col("state") == 'NV') & (col("city") == 'North Las Vegas')).groupBy("name", "rev_stars").count().show()



+--------------------+---------+-----+
|                name|rev_stars|count|
+--------------------+---------+-----+
|Pips Cucina & Win...|      4.0|    2|
|Perfect Tint & Se...|      5.0|    3|
|      The Shade Tree|      5.0|    6|
|GEICO Insurance A...|      4.0|    1|
| Attack Pest Control|      5.0|    7|
|Batteries Plus Bulbs|      4.0|    4|
|        Golden Spoon|      5.0|    1|
|  Jack's Tax Service|      5.0|   18|
|Korner Kwik Lube ...|      5.0|   32|
|  Paws On the Ground|      5.0|    4|
|REAL RESULTS Fitn...|      5.0|    5|
|Tule Springs Foss...|      4.0|    1|
|Skip's Puppy Poo-...|      4.0|    1|
|          PrideStaff|      5.0|    2|
|Alexander Dental ...|      4.0|    3|
|       Go For It USA|      4.0|    4|
|Ted Wiens Tire & ...|      4.0|    4|
|Vision Source - N...|      4.0|    1|
|                IHOP|      5.0|   20|
|Barrett Pool and ...|      5.0|    2|
+--------------------+---------+-----+
only showing top 20 rows



In [55]:
df_business.select("categories").groupBy("categories").count().show(100, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|categories                                                                                                                                       |count|
+-------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Health & Medical, Dentists, Oral Surgeons, Doctors, Cosmetic Dentists                                                                            |20   |
|Music & DVDs, Videos & Video Game Rental, Vinyl Records, Books, Mags, Music & Video, Shopping                                                    |3    |
|Parks, Active Life, Dog Parks                                                                                                                    |252  |
|Breakfast & Brunch, Burgers, Sandwiches, Restaurants, American (Traditional

In [60]:
df_business.where(col("categories").like("%Auto%")).select("categories").groupBy("categories").count().show(100, False)



+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|categories                                                                                                                                                                             |count|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Shopping, Sporting Goods, Local Services, Automotive, Motorcycle Dealers, Bikes, Bike Repair/Maintenance                                                                               |6    |
|Automotive, Car Dealers, Auto Parts & Supplies, Auto Repair                                                                                                                            |1207 |
|Electronics, Shopping, Home Theatre Ins

In [66]:
df_business.select("attributes.RestaurantsPriceRange2").groupBy("RestaurantsPriceRange2").count().show(100, False)


+----------------------+-------+
|RestaurantsPriceRange2|count  |
+----------------------+-------+
|3                     |523610 |
|None                  |677    |
|null                  |1177506|
|1                     |1355845|
|4                     |100846 |
|2                     |3527416|
+----------------------+-------+



In [70]:
df_business.filter("attributes.RestaurantsPriceRange2 == 'None'").select("attributes.RestaurantsPriceRange2","categories").groupBy("categories").count().show(100, False)



+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|categories                                                                                                                                                                                    |count|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Nightlife, Dive Bars, Restaurants, American (Traditional), Bars                                                                                                                               |4    |
|Food, Ice Cream & Frozen Yogurt                                                                                                                                                               |5    |
|Beau